https://bkshin.tistory.com/entry/OpenCV-9-%EC%9D%B4%EB%AF%B8%EC%A7%80-%EC%97%B0%EC%82%B0

4:3 and 16:9

In [71]:
import requests as rq
from io import BytesIO
from tqdm import tqdm

In [72]:
from PIL import Image
from glob import glob
import numpy as np
import cv2
import sys
from  PIL  import Image

In [73]:
import matplotlib.pylab as plt
from xml.etree.ElementTree import Element, SubElement, ElementTree, dump

In [74]:
# object_loc = 'C:/Users/multicampus/projects/fresh/data/Background/zz.jpg'

## Import image
- [B, G, R]로 읽음

In [75]:
# object_loc = 'C:/Users/multicampus/projects/fresh/data/Training/apple_fuji/apple_fuji_L/apple_fuji_L_1-1_1TOP.png'

# object_loc = 'C:/Users/multicampus/projects/fresh/data/Training/garlic_uiseong/garlic_uiseong_L/garlic_uiseong_L_1-2_2FR45.png'

# object_loc = 'C:/Users/multicampus/projects/fresh/data/Training/cabbage_red/cabbage_red_L/cabbage_red_L_1-3_3FR90.png'

# background_loc = 'C:/Users/multicampus/projects/fresh/data/Background/mart_1.jpg'

In [76]:
def show_img(*args):
    for idx, img in enumerate(args):
        cv2.imshow(f'winname{idx}', img)
    cv2.waitKey()
    cv2.destroyAllWindows()

## 배경 제거 - 색깔(크로마키)

In [77]:
def object_locate(object_img, background_img):
    height1, width1 = object_img.shape[:2]
    height2, width2 = background_img.shape[:2]
    x1 = int((width2 - width1) * np.random.rand())
    y1 = int((height2 - height1) * np.random.rand())
    x2 = x1 + width1
    y2 = y1 + height1
    return x1, y1, x2, y2

In [78]:
def get_hsv(object_img):
    chromakey = object_img[:10, :10, :]
    hsv_chroma = cv2.cvtColor(chromakey, cv2.COLOR_BGR2HSV)
    hsv_img = cv2.cvtColor(object_img, cv2.COLOR_BGR2HSV)
    chroma_h = hsv_chroma[:,:,0]
    return hsv_img

In [79]:
def remove_bg1(object_img, offset=15):
    chromakey = np.concatenate((object_img[:10, :10, :], object_img[:10, -10:, :],
                      object_img[-10:, :10, :], object_img[-10:, -10:, :]))
    hsv_chroma = cv2.cvtColor(chromakey, cv2.COLOR_BGR2HSV)
    hsv_img = cv2.cvtColor(object_img, cv2.COLOR_BGR2HSV)
    chroma_h = hsv_chroma[:,:,0]
    chroma_s = hsv_chroma[:, :, 1]
    chroma_v = hsv_chroma[:, :, 2]
    
    lower = np.array([chroma_h.min()-offset, max(0, chroma_s.min()-offset), 100])
    upper = np.array([chroma_h.max()+offset, min(255, chroma_s.max()+offset), 255])
    #--⑥ 마스크 생성 및 마스킹 후 합성
    mask = cv2.inRange(hsv_img, lower, upper)
    mask = cv2.bitwise_not(mask)
    return mask

In [80]:
def remove_bg2(object_img):
    l = 20
    u = 40
    ed = cv2.cvtColor(object_img, cv2.COLOR_BGR2GRAY)
    edges = cv2.GaussianBlur(object_img, (21, 51), 3)
    edges = cv2.cvtColor(edges, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(edges, l, u)
    _, thresh = cv2.threshold(edges, 0, 255, cv2.THRESH_BINARY  + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=4)
    return mask

In [81]:
def mask_fill(mask):
    data = mask.tolist()
    sys.setrecursionlimit(10**8)
    for i in  range(len(data)):
        for j in  range(len(data[i])):
            if data[i][j] !=  255:
                data[i][j] =  -1
            else:
                break
        for j in  range(len(data[i])-1, -1, -1):
            if data[i][j] !=  255:
                data[i][j] =  -1
            else:
                break
    image = np.array(data)
    image[image !=  -1] =  255
    image[image ==  -1] =  0
    mask = np.array(image, np.uint8)
    return mask

In [82]:
def overlap_square(rect1, rect2):
    return not (rect1[2] < rect2[0] or rect1[0] > rect2[2] or rect1[1] > rect2[3] or rect1[3] < rect2[1])

In [83]:
def overlap(object_img, background_img, mask, loc):
    mask_inv = cv2.bitwise_not(mask)
#     background_img = background_img.copy()
    x, y, w, h = loc
    roi = background_img[y:h, x:w]
    fg = cv2.bitwise_and(object_img, object_img, mask=mask)
    bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
    background_img[y:h, x:w] = fg + bg
    return background_img

In [84]:
def indent(elem, level=0): #자료 출처 https://goo.gl/J8VoDK
    i = "\n" + level*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

In [85]:
def make_xml(filename, background_img, visited, target_names):
    height = background_img.shape[0]
    width = background_img.shape[1]

    root = Element('annotation')
    SubElement(root, 'folder').text = 'custom_images'
    SubElement(root, 'filename').text = filename
    SubElement(root, 'path').text = f'{DATA_ROOT_FOLDER}/images/{filename}'
    source = SubElement(root, 'source')
    SubElement(source, 'database').text = 'Unknown'

    size = SubElement(root, 'size')
    SubElement(size, 'width').text = str(width)
    SubElement(size, 'height').text = str(height)
    SubElement(size, 'depth').text = '1'

    SubElement(root, 'segmented').text = '0'

    for i in range(len(visited)):
        label = target_names[i]
        x1, y1, x2, y2 = visited[i]
        point1 = (x1, y1)
        point2 = (x2, y2)
        obj = SubElement(root, 'object')
        SubElement(obj, 'name').text = label
        SubElement(obj, 'pose').text = 'Unspecified'
        SubElement(obj, 'truncated').text = '0'
        SubElement(obj, 'difficult').text = '0'
        bbox = SubElement(obj, 'bndbox')
        SubElement(bbox, 'xmin').text = str(point1[0])
        SubElement(bbox, 'ymin').text = str(point1[1])
        SubElement(bbox, 'xmax').text = str(point2[0])
        SubElement(bbox, 'ymax').text = str(point2[1])
    indent(root)
    tree = ElementTree(root)
    return tree

In [86]:
DATA_ROOT_FOLDER = 'C:/Users/multicampus/projects/fresh/data/overlap'

In [87]:
img_locations = {}

In [88]:
folders = [x for x in glob('C:/Users/multicampus/projects/fresh/data/Training/*') if '[라벨]' not in x]
for f in folders:
    target = f.split('\\')[1]
    targets = [target + '_L', target + '_M', target + '_S']
    for t in targets:
        img_locations[t] = glob(f'{f}/{t}/*')

In [89]:
targets = list(img_locations.keys())

In [90]:
# background_loc = 'C:/Users/multicampus/projects/fresh/data/Background/mart_1.jpg'
# background_img = cv2.imread(background_loc)

In [91]:
import time

In [224]:
for idx in tqdm(range(1, 201)):
    w = np.random.choice(list(range(1152, 1408 + 1, 32)))
    h = w * (1 + np.random.random() * (1/2.6))  # w:h = 3:4 정도가 되게
    h = int((h // 32) * 32)
    res = rq.get(f'https://picsum.photos/{w}/{h}')
    res = Image.open(BytesIO(res.content))
    background_img = np.array(res)

#     maximum_size = int(min(background_img.shape[:2]) / 2)
#     minimum_size = int(min(background_img.shape[:2]) / 9)

    visited = []
    target_names = []

    for i in range(10):
        target = np.random.choice(targets)
        object_loc = np.random.choice(img_locations[target])
#         size = min(1000, int(minimum_size + maximum_size*np.random.random()))  # 최대 크기는 1000
#         size = min(1000, int(600 + 450*np.random.random()))
        object_img = cv2.imread(object_loc)
        object_img = cv2.resize(object_img, dsize=(size, size), interpolation=cv2.INTER_AREA)
        count = 10
        while count:
            rec_new = object_locate(object_img, background_img)
            x1_new, y1_new, x2_new, y2_new = rec_new
            flag = 0
            for rec in visited:
                if overlap_square(rec, rec_new):  # 겹쳐지면
                    count -= 1
                    break
            else:
                break


        if count:  # 5번 시도만에 적절한 위치를 찾은 경우
            mask = remove_bg1(object_img, 15)
            mask = mask_fill(mask)
            result = overlap(object_img, background_img, mask, rec_new)
            visited.append(rec_new)
#             target_names.append(target)
            target_names.append(target[:-2])  # 18개종
        else:
            pass
    #         print('end')

    image_save_folder = 'C:/Users/multicampus/projects/fresh/data/overlap/images'

    cv2.imwrite(f'{DATA_ROOT_FOLDER}/images/img_{idx}.png', background_img)

    tree = make_xml(f'img_{idx}.png', background_img, visited, target_names)

    tree.write(f'{DATA_ROOT_FOLDER}/annotations/img_{idx}.xml')
    time.sleep(5)

  0%|          | 1/200 [00:09<30:05,  9.08s/it]


KeyboardInterrupt: 

In [35]:
targets_ = targets.copy()

In [39]:
targets_ = list(set([x[:-2] for x in targets_]))

In [40]:
f = open("./labels_ag_trimmed.txt", 'w')
for t in targets_[:-1]:
    data = f'{t}\n'
    f.write(data)
f.write(f'{targets_[-1]}')
f.close()

In [985]:
len(targets)

54

In [528]:
# v2
mask = remove_bg2(object_img)
mask = mask_fill(mask)
result = overlap(object_img, background_img, mask)
show_img(result)

## 배경제거 - 객체 가장자리
https://dev.to/stokry/remove-background-from-images-with-python-dkj

In [218]:
object_loc = 'C:/Users/multicampus/projects/fresh/data/Training/apple_fuji/apple_fuji_L/apple_fuji_L_1-1_1TOP.png'

In [253]:
object_loc = 'C:/Users/multicampus/projects/fresh/data/Training/garlic_uiseong/garlic_uiseong_L/garlic_uiseong_L_1-2_2FR45.png'

In [254]:
object_img = cv2.imread(object_loc, cv2.IMREAD_UNCHANGED)

In [255]:
object_img = cv2.resize(object_img, dsize=(200, 200), interpolation=cv2.INTER_AREA)
# original = img.copy()
# original = cv2.resize(original, dsize=(300, 300), interpolation=cv2.INTER_AREA)

In [256]:
background_img = cv2.imread(background_loc)

In [344]:
mask = remove_bg2(object_img)

In [346]:
mask = mask_fill(mask)

In [347]:
mask_inv = cv2.bitwise_not(mask)

In [358]:
result = cv2.bitwise_and(object_img, object_img, mask=mask)

In [359]:
show_img(result)

In [348]:
# overlap
x, y, w, h = object_locate(object_img, background_img)
roi = background_img[y:h, x:w]
fg = cv2.bitwise_and(object_img, object_img, mask=mask)
bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
background_img[y:h, x:w] = fg + bg

In [350]:
show_img(background_img)

In [351]:
result = cv2.bitwise_and(object_img, object_img, mask=mask)

In [355]:
show_img(result)

In [354]:
mask ==  0

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [356]:
result = cv2.bitwise_and(object_img, object_img, mask=mask)
result[mask ==  0] =  255

In [357]:
show_img(result)

In [143]:
cv2.imwrite('bg.png', result)

img = Image.open('bg.png')
img.convert("RGBA")
datas = img.getdata()

newData = []
for item in datas:
    if item[0] ==  255  and item[1] ==  255  and item[2] ==  255:
        newData.append((255, 255, 255, 0))
    else:
        newData.append(item)

img.putdata(newData)
# img.save("img.png", "PNG")

In [145]:
show_img(result)

In [45]:
mart.paste(apple, area)

## HSV로 안바꾸고 지우기

In [115]:
mart = Image.open(background_loc)
mart = mart.resize((500, 500))
print(mart.format, mart.size, mart.mode)

None (500, 500) RGB


In [116]:
# apple = Image.open(f'{object_loc}/apple_fuji/apple_fuji_L/apple_fuji_L_1-1_1TOP.png')
apple = Image.open(object_loc)
apple = apple.resize((200, 200))
print(apple.format, apple.size, apple.mode)
datas = apple.getdata()
# print(datas)

None (200, 200) RGB


In [117]:
# 배경 제거
newData = []
cutOff = 180

for item in datas:
    # print(item)
    if item[0] >= cutOff and item[1] >= cutOff and item[2] >= cutOff:
        # print('chk')
        newData.append((0, 0,0))
    else:
        newData.append(item)

apple.putdata(newData)

area = (5, 5)
mart.paste(apple, area)

mart.show()